In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Phase 1: Data Exploration & Cleaning\n",
    "## Amazon Delivery Time Prediction Project\n",
    "\n",
    "This notebook covers:\n",
    "1. Data Loading and Initial Inspection\n",
    "2. Data Quality Assessment\n",
    "3. Data Cleaning and Preprocessing\n",
    "4. Exploratory Data Analysis (EDA)\n",
    "5. Outlier Detection and Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Import required libraries\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "import warnings\n",
    "warnings.filterwarnings('ignore')\n",
    "\n",
    "# Set display options\n",
    "pd.set_option('display.max_columns', None)\n",
    "pd.set_option('display.max_rows', 100)\n",
    "\n",
    "# Set plotting style\n",
    "sns.set_style('whitegrid')\n",
    "plt.rcParams['figure.figsize'] = (12, 6)\n",
    "\n",
    "print('✓ Libraries imported successfully!')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Data Loading and Initial Inspection"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load the dataset\n",
    "df = pd.read_csv('../data/raw/amazon_delivery.csv')\n",
    "\n",
    "print(f\"Dataset loaded successfully!\")\n",
    "print(f\"Shape: {df.shape}\")\n",
    "print(f\"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Display first few rows\n",
    "print(\"First 5 rows of the dataset:\")\n",
    "df.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Display dataset info\n",
    "print(\"Dataset Information:\")\n",
    "df.info()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Display statistical summary\n",
    "print(\"Statistical Summary:\")\n",
    "df.describe()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Data Quality Assessment"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Check for missing values\n",
    "print(\"Missing Values Analysis:\")\n",
    "missing_data = pd.DataFrame({\n",
    "    'Column': df.columns,\n",
    "    'Missing_Count': df.isnull().sum().values,\n",
    "    'Missing_Percentage': (df.isnull().sum().values / len(df) * 100).round(2)\n",
    "})\n",
    "\n",
    "missing_data = missing_data[missing_data['Missing_Count'] > 0].sort_values(\n",
    "    'Missing_Percentage', ascending=False\n",
    ").reset_index(drop=True)\n",
    "\n",
    "if not missing_data.empty:\n",
    "    print(missing_data)\n",
    "else:\n",
    "    print(\"✓ No missing values found!\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Check for duplicate rows\n",
    "duplicate_count = df.duplicated().sum()\n",
    "duplicate_pct = (duplicate_count / len(df)) * 100\n",
    "\n",
    "print(f\"Duplicate Rows Analysis:\")\n",
    "print(f\"  Total duplicates: {duplicate_count} ({duplicate_pct:.2f}%)\")\n",
    "\n",
    "if duplicate_count > 0:\n",
    "    print(f\"  ⚠️ Found {duplicate_count} duplicate rows\")\n",
    "else:\n",
    "    print(\"  ✓ No duplicate rows found\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Check unique values for categorical columns\n",
    "print(\"Categorical Columns Analysis:\")\n",
    "categorical_cols = df.select_dtypes(include=['object']).columns\n",
    "\n",
    "for col in categorical_cols:\n",
    "    unique_count = df[col].nunique()\n",
    "    print(f\"\\n{col}:\")\n",
    "    print(f\"  Unique values: {unique_count}\")\n",
    "    print(f\"  Values: {df[col].unique()[:10].tolist()}\")\n",
    "    if unique_count > 10:\n",
    "        print(f\"  ... and {unique_count - 10} more\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Data Cleaning"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create a copy for cleaning\n",
    "df_clean = df.copy()\n",
    "print(f\"Original shape: {df.shape}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Step 1: Remove whitespace from string columns\n",
    "print(\"Step 1: Removing whitespace...\")\n",
    "for col in df_clean.select_dtypes(include=['object']).columns:\n",
    "    df_clean[col] = df_clean[col].str.strip()\n",
    "print(\"✓ Whitespace removed\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Step 2: Standardize categorical values\n",
    "print(\"Step 2: Standardizing categorical values...\")\n",
    "\n",
    "# Fix Area column spelling\n",
    "df_clean['Area'] = df_clean['Area'].replace({\n",
    "    'Metropolitian': 'Metropolitan'\n",
    "})\n",
    "\n",
    "# Check unique values after standardization\n",
    "print(\"\\nUnique values after standardization:\")\n",
    "for col in ['Weather', 'Traffic', 'Vehicle', 'Area']:\n",
    "    print(f\"{col}: {df_clean[col].unique().tolist()}\")\n",
    "\n",
    "print(\"✓ Categories standardized\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Step 3: Remove duplicates\n",
    "print(\"Step 3: Removing duplicates...\")\n",
    "rows_before = len(df_clean)\n",
    "df_clean.drop_duplicates(inplace=True)\n",
    "rows_after = len(df_clean)\n",
    "print(f\"✓ Removed {rows_before - rows_after} duplicate rows\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Step 4: Handle missing values\n",
    "print(\"Step 4: Handling missing values...\")\n",
    "\n",
    "# For numerical columns: use median\n",
    "numerical_cols = df_clean.select_dtypes(include=[np.number]).columns\n",
    "for col in numerical_cols:\n",
    "    if df_clean[col].isnull().sum() > 0:\n",
    "        df_clean[col].fillna(df_clean[col].median(), inplace=True)\n",
    "        print(f\"  ✓ Filled {col} with median\")\n",
    "\n",
    "# For categorical columns: use mode\n",
    "categorical_cols = df_clean.select_dtypes(include=['object']).columns\n",
    "for col in categorical_cols:\n",
    "    if df_clean[col].isnull().sum() > 0:\n",
    "        mode_value = df_clean[col].mode()[0] if not df_clean[col].mode().empty else 'Unknown'\n",
    "        df_clean[col].fillna(mode_value, inplace=True)\n",
    "        print(f\"  ✓ Filled {col} with mode\")\n",
    "\n",
    "print(f\"\\nMissing values after cleaning: {df_clean.isnull().sum().sum()}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Step 5: Convert data types\n",
    "print(\"Step 5: Converting data types...\")\n",
    "\n",
    "# Convert Order_Date to datetime\n",
    "df_clean['Order_Date'] = pd.to_datetime(df_clean['Order_Date'])\n",
    "print(\"✓ Converted Order_Date to datetime\")\n",
    "\n",
    "# Convert time columns\n",
    "df_clean['Order_Time'] = pd.to_datetime(df_clean['Order_Time'], format='%H:%M:%S').dt.time\n",
    "df_clean['Pickup_Time'] = pd.to_datetime(df_clean['Pickup_Time'], format='%H:%M:%S').dt.time\n",
    "print(\"✓ Converted time columns\")\n",
    "\n",
    "print(f\"\\nCleaned dataset shape: {df_clean.shape}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Exploratory Data Analysis (EDA)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 4.1 Target Variable Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Delivery Time distribution\n",
    "fig, axes = plt.subplots(1, 3, figsize=(18, 5))\n",
    "\n",
    "# Histogram\n",
    "axes[0].hist(df_clean['Delivery_Time'], bins=50, edgecolor='black', alpha=0.7)\n",
    "axes[0].set_title('Delivery Time Distribution', fontsize=14, fontweight='bold')\n",
    "axes[0].set_xlabel('Delivery Time (minutes)')\n",
    "axes[0].set_ylabel('Frequency')\n",
    "\n",
    "# Box plot\n",
    "axes[1].boxplot(df_clean['Delivery_Time'])\n",
    "axes[1].set_title('Delivery Time Box Plot', fontsize=14, fontweight='bold')\n",
    "axes[1].set_ylabel('Delivery Time (minutes)')\n",
    "\n",
    "# Q-Q plot\n",
    "from scipy import stats\n",
    "stats.probplot(df_clean['Delivery_Time'], dist=\"norm\", plot=axes[2])\n",
    "axes[2].set_title('Q-Q Plot', fontsize=14, fontweight='bold')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "# Statistics\n",
    "print(\"\\nDelivery Time Statistics:\")\n",
    "print(df_clean['Delivery_Time'].describe())"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 4.2 Numerical Features Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Distribution of numerical features\n",
    "numerical_features = ['Agent_Age', 'Agent_Rating', 'Delivery_Time']\n",
    "\n",
    "fig, axes = plt.subplots(1, 3, figsize=(18, 5))\n",
    "\n",
    "for idx, col in enumerate(numerical_features):\n",
    "    axes[idx].hist(df_clean[col], bins=30, edgecolor='black', alpha=0.7, color='steelblue')\n",
    "    axes[idx].set_title(f'Distribution of {col}', fontsize=12, fontweight='bold')\n",
    "    axes[idx].set_xlabel(col)\n",
    "    axes[idx].set_ylabel('Frequency')\n",
    "    \n",
    "    # Add mean and median lines\n",
    "    mean_val = df_clean[col].mean()\n",
    "    median_val = df_clean[col].median()\n",
    "    axes[idx].axvline(mean_val, color='red', linestyle='--', linewidth=2, label=f'Mean: {mean_val:.2f}')\n",
    "    axes[idx].axvline(median_val, color='green', linestyle='--', linewidth=2, label=f'Median: {median_val:.2f}')\n",
    "    axes[idx].legend()\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 4.3 Categorical Features Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Distribution of categorical features\n",
    "categorical_features = ['Weather', 'Traffic', 'Vehicle', 'Area']\n",
    "\n",
    "fig, axes = plt.subplots(2, 2, figsize=(16, 12))\n",
    "axes = axes.flatten()\n",
    "\n",
    "for idx, col in enumerate(categorical_features):\n",
    "    value_counts = df_clean[col].value_counts()\n",
    "    axes[idx].bar(range(len(value_counts)), value_counts.values, \n",
    "                  color='coral', edgecolor='black')\n",
    "    axes[idx].set_xticks(range(len(value_counts)))\n",
    "    axes[idx].set_xticklabels(value_counts.index, rotation=45, ha='right')\n",
    "    axes[idx].set_title(f'Distribution of {col}', fontsize=12, fontweight='bold')\n",
    "    axes[idx].set_xlabel(col)\n",
    "    axes[idx].set_ylabel('Count')\n",
    "    \n",
    "    # Add count labels\n",
    "    for i, v in enumerate(value_counts.values):\n",
    "        axes[idx].text(i, v + max(value_counts.values)*0.01, str(v), \n",
    "                      ha='center', va='bottom', fontweight='bold')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 4.4 Bivariate Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Delivery Time by categorical features\n",
    "fig, axes = plt.subplots(2, 2, figsize=(16, 12))\n",
    "axes = axes.flatten()\n",
    "\n",
    "for idx, col in enumerate(categorical_features):\n",
    "    df_clean.boxplot(column='Delivery_Time', by=col, ax=axes[idx])\n",
    "    axes[idx].set_title(f'Delivery Time by {col}', fontsize=12, fontweight='bold')\n",
    "    axes[idx].set_xlabel(col)\n",
    "    axes[idx].set_ylabel('Delivery Time (minutes)')\n",
    "    plt.suptitle('')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "# Print statistics
print("\\nAverage Delivery Time by Category:\")\n",
    "for col in categorical_features:\n",
    "    print(f\"\\n{col}:\")\n",
    "    stats_df = df_clean.groupby(col)['Delivery_Time'].agg(['mean', 'median', 'count'])\n",
    "    print(stats_df.sort_values('mean', ascending=False))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Correlation analysis\n",
    "numerical_cols = df_clean.select_dtypes(include=[np.number]).columns\n",
    "corr_matrix = df_clean[numerical_cols].corr()\n",
    "\n",
    "plt.figure(figsize=(10, 8))\n",
    "sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', \n",
    "            center=0, square=True, linewidths=1)\n",
    "plt.title('Correlation Heatmap', fontsize=16, fontweight='bold')\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "# Print high correlations\n",
    "print(\"\\nHigh correlations with Delivery_Time (|r| > 0.3):\")\n",
    "delivery_corr = corr_matrix['Delivery_Time'].sort_values(ascending=False)\n",
    "print(delivery_corr[abs(delivery_corr) > 0.3])"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 4.5 Time-Based Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Extract time features\n",
    "df_clean['Day_of_Week'] = df_clean['Order_Date'].dt.day_name()\n",
    "df_clean['Month'] = df_clean['Order_Date'].dt.month_name()\n",
    "df_clean['Hour'] = df_clean['Order_Date'].apply(lambda x: pd.to_datetime(str(x)).hour)\n",
    "\n",
    "fig, axes = plt.subplots(2, 2, figsize=(16, 12))\n",
    "\n",
    "# Orders by day of week\n",
    "day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']\n",
    "day_counts = df_clean['Day_of_Week'].value_counts().reindex(day_order)\n",
    "axes[0, 0].bar(range(len(day_counts)), day_counts.values, color='skyblue', edgecolor='black')\n",
    "axes[0, 0].set_xticks(range(len(day_counts)))\n",
    "axes[0, 0].set_xticklabels(day_order, rotation=45, ha='right')\n",
    "axes[0, 0].set_title('Orders by Day of Week', fontsize=14, fontweight='bold')\n",
    "axes[0, 0].set_ylabel('Count')\n",
    "\n",
    "# Average delivery time by day of week\n",
    "day_delivery = df_clean.groupby('Day_of_Week')['Delivery_Time'].mean().reindex(day_order)\n",
    "axes[0, 1].bar(range(len(day_delivery)), day_delivery.values, color='coral', edgecolor='black')\n",
    "axes[0, 1].set_xticks(range(len(day_delivery)))\n",
    "axes[0, 1].set_xticklabels(day_order, rotation=45, ha='right')\n",
    "axes[0, 1].set_title('Avg Delivery Time by Day', fontsize=14, fontweight='bold')\n",
    "axes[0, 1].set_ylabel('Avg Delivery Time (min)')\n",
    "\n",
    "# Orders over time\n",
    "daily_orders = df_clean.groupby('Order_Date').size()\n",
    "axes[1, 0].plot(daily_orders.index, daily_orders.values, color='green', linewidth=2)\n",
    "axes[1, 0].set_title('Orders Over Time', fontsize=14, fontweight='bold')\n",
    "axes[1, 0].set_xlabel('Date')\n",
    "axes[1, 0].set_ylabel('Number of Orders')\n",
    "axes[1, 0].tick_params(axis='x', rotation=45)\n",
    "\n",
    "# Product category distribution\n",
    "category_counts = df_clean['Category'].value_counts().head(10)\n",
    "axes[1, 1].barh(range(len(category_counts)), category_counts.values, color='purple')\n",
    "axes[1, 1].set_yticks(range(len(category_counts)))\n",
    "axes[1, 1].set_yticklabels(category_counts.index)\n",
    "axes[1, 1].set_title('Top 10 Product Categories', fontsize=14, fontweight='bold')\n",
    "axes[1, 1].set_xlabel('Count')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Outlier Detection and Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Function to detect outliers using IQR\n",
    "def detect_outliers_iqr(data, column, threshold=1.5):\n",
    "    Q1 = data[column].quantile(0.25)\n",
    "    Q3 = data[column].quantile(0.75)\n",
    "    IQR = Q3 - Q1\n",
    "    lower_bound = Q1 - threshold * IQR\n",
    "    upper_bound = Q3 + threshold * IQR\n",
    "    outliers = (data[column] < lower_bound) | (data[column] > upper_bound)\n",
    "    return outliers, lower_bound, upper_bound\n",
    "\n",
    "# Analyze outliers for key numerical columns\n",
    "outlier_cols = ['Agent_Age', 'Agent_Rating', 'Delivery_Time']\n",
    "\n",
    "fig, axes = plt.subplots(len(outlier_cols), 2, figsize=(16, 12))\n",
    "\n",
    "for idx, col in enumerate(outlier_cols):\n",
    "    outliers, lower, upper = detect_outliers_iqr(df_clean, col)\n",
    "    \n",
    "    # Box plot\n",
    "    axes[idx, 0].boxplot(df_clean[col])\n",
    "    axes[idx, 0].axhline(y=lower, color='r', linestyle='--', label=f'Lower: {lower:.2f}')\n",
    "    axes[idx, 0].axhline(y=upper, color='r', linestyle='--', label=f'Upper: {upper:.2f}')\n",
    "    axes[idx, 0].set_title(f'Box Plot: {col}', fontsize=12, fontweight='bold')\n",
    "    axes[idx, 0].set_ylabel(col)\n",
    "    axes[idx, 0].legend()\n",
    "    \n",
    "    # Histogram with outliers highlighted\n",
    "    axes[idx, 1].hist(df_clean[col], bins=50, alpha=0.7, color='skyblue', edgecolor='black')\n",
    "    if outliers.sum() > 0:\n",
    "        axes[idx, 1].hist(df_clean[outliers][col], bins=20, alpha=0.7, \n",
    "                         color='red', edgecolor='black', label='Outliers')\n",
    "    axes[idx, 1].set_title(f'Distribution: {col}', fontsize=12, fontweight='bold')\n",
    "    axes[idx, 1].set_xlabel(col)\n",
    "    axes[idx, 1].set_ylabel('Frequency')\n",
    "    axes[idx, 1].legend()\n",
    "    \n",
    "    print(f\"\\n{col}:\")\n",
    "    print(f\"  Outliers: {outliers.sum()} ({outliers.sum()/len(df_clean)*100:.2f}%)\")\n",
    "    print(f\"  Bounds: [{lower:.2f}, {upper:.2f}]\")\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Key Insights Summary"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"=\"*70)\n",
    "print(\"KEY INSIGHTS FROM PHASE 1 ANALYSIS\")\n",
    "print(\"=\"*70)\n",
    "\n",
    "print(\"\\n1. DATASET OVERVIEW:\")\n",
    "print(f\"   • Total records: {len(df_clean):,}\")\n",
    "print(f\"   • Features: {len(df_clean.columns)}\")\n",
    "print(f\"   • Date range: {df_clean['Order_Date'].min()} to {df_clean['Order_Date'].max()}\")\n",
    "\n",
    "print(\"\\n2. DELIVERY TIME INSIGHTS:\")\n",
    "print(f\"   • Average: {df_clean['Delivery_Time'].mean():.2f} minutes\")\n",
    "print(f\"   • Median: {df_clean['Delivery_Time'].median():.2f} minutes\")\n",
    "print(f\"   • Range: {df_clean['Delivery_Time'].min():.0f} - {df_clean['Delivery_Time'].max():.0f} minutes\")\n",
    "print(f\"   • Std Dev: {df_clean['Delivery_Time'].std():.2f} minutes\")\n",
    "\n",
    "print(\"\\n3. WEATHER IMPACT:\")\n",
    "weather_impact = df_clean.groupby('Weather')['Delivery_Time'].mean().sort_values(ascending=False)\n",
    "for weather, time in weather_impact.items():\n",
    "    print(f\"   • {weather}: {time:.2f} min\")\n",
    "\n",
    "print(\"\\n4. TRAFFIC IMPACT:\")\n",
    "traffic_impact = df_clean.groupby('Traffic')['Delivery_Time'].mean().sort_values(ascending=False)\n",
    "for traffic, time in traffic_impact.items():\n",
    "    print(f\"   • {traffic}: {time:.2f} min\")\n",
    "\n",
    "print(\"\\n5. VEHICLE EFFICIENCY:\")\n",
    "vehicle_impact = df_clean.groupby('Vehicle')['Delivery_Time'].mean().sort_values()\n",
    "for vehicle, time in vehicle_impact.items():\n",
    "    print(f\"   • {vehicle}: {time:.2f} min\")\n",
    "\n",
    "print(\"\\n6. AREA ANALYSIS:\")\n",
    "area_impact = df_clean.groupby('Area')['Delivery_Time'].mean().sort_values(ascending=False)\n",
    "for area, time in area_impact.items():\n",
    "    print(f\"   • {area}: {time:.2f} min\")\n",
    "\n",
    "print(\"\\n7. DATA QUALITY:\")\n",
    "print(f\"   • Missing values: {df_clean.isnull().sum().sum()}\")\n",
    "print(f\"   • Duplicate rows: {df_clean.duplicated().sum()}\")\n",
    "print(f\"   • Data types: All converted appropriately\")\n",
    "\n",
    "print(\"\\n\" + \"=\"*70)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Save Cleaned Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Save cleaned dataset\n",
    "output_path = '../data/processed/cleaned_data.csv'\n",
    "df_clean.to_csv(output_path, index=False)\n",
    "print(f\"✅ Cleaned data saved to: {output_path}\")\n",
    "print(f\"\\nCleaned dataset shape: {df_clean.shape}\")\n",
    "print(f\"Records removed: {len(df) - len(df_clean)}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 8. Next Steps\n",
    "\n",
    "Based on the analysis, the following steps are recommended for **Phase 2: Feature Engineering**:\n",
    "\n",
    "### Feature Engineering Tasks:\n",
    "1. **Geospatial Features**:\n",
    "   - Calculate distance between store and drop locations using Haversine formula\n",
    "   - Consider creating distance bins (short, medium, long)\n",
    "\n",
    "2. **Time-Based Features**:\n",
    "   - Extract hour of day, day of week, is_weekend\n",
    "   - Calculate time difference between order and pickup\n",
    "   - Create time_of_day categories (morning, afternoon, evening, night)\n",
    "\n",
    "3. **Interaction Features**:\n",
    "   - Traffic × Distance\n",
    "   - Weather × Vehicle type\n",
    "   - Area × Time of day\n",
    "\n",
    "4. **Encoding Strategies**:\n",
    "   - One-Hot Encoding for: Vehicle, Area (low cardinality)\n",
    "   - Target Encoding for: Category (high cardinality)\n",
    "   - Label Encoding for: Weather, Traffic (ordinal nature)\n",
    "\n",
    "5. **Feature Scaling**:\n",
    "   - StandardScaler for distance features\n",
    "   - MinMaxScaler for Agent_Age and Agent_Rating\n",
    "\n",
    "### Key Observations for Modeling:\n",
    "- **Strong predictors**: Traffic, Weather, Vehicle type, Distance\n",
    "- **Target variable**: Slightly right-skewed, may benefit from log transformation\n",
    "- **Outliers**: Present but capped to reasonable ranges\n",
    "- **Missing values**: All handled appropriately\n",
    "\n",
    "### Recommended Models for Phase 5:\n",
    "1. **Baseline**: Linear Regression\n",
    "2. **Tree-based**: Random Forest, Gradient Boosting\n",
    "3. **Advanced**: XGBoost, LightGBM\n",
    "4. **Ensemble**: Stacking/Voting Regressor"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}